# S-N-Annealed-VAE Model Testing Framework

In [ ]:
import sys
from pathlib import Path

# Find project root by looking for .git or requirements.txt
current = Path.cwd()
while not any((current / marker).exists() for marker in ['.git', 'requirements.txt']):
    if current.parent == current:
        raise FileNotFoundError("Project root not found")
    current = current.parent

sys.path.append(str(current))
print(f"Added project root: {current}")

# Hyperparameters

In [ ]:
import torch

#### deterministic run ####
determinism_kwargs = {
    'seed':0,
    'use_cuda_det': True,
    'enforce_det':False,
    'cublas_workspace_config': None,
}

##### Model parameters #####
model_name = 's_n_vae_locatello'  # S-N-VAE model with Locatello architecture
model_decoder_output_dist = 'bernoulli'  # Output distribution of the decoder

# Define latent factor topologies: mix of Normal (R1) and Power Spherical (S1)
latent_factor_topologies = ['S1', 'R1', 'R1']  # 3 factors total

use_torch_compile = False  # Use torch.compile for model compilation

#### Training parameters ####
train_step_unit = 'epoch'  # Unit for training steps ('epoch' or 'iteration')
num_train_steps = 300

# train_step_unit = 'iteration'  # Unit for training steps ('epoch' or 'iteration')
# num_train_steps = int(9e3)  # Number of training steps 

learning_rate = 1e-4
batch_size = 64

#### losses ####
loss_name = 'anneal_s_n_vae'      # S-N-VAE Annealed loss
loss_kwargs_dsprites = {
    'C_init': 0.0,              # Starting annealed capacity
    'C_fin': 5.0,               # Final annealed capacity
    'gamma': 100.0,             # Weight of the KL divergence term
    'anneal_steps': 50000,      # Number of steps for annealing (~2/3 of total training)
    'latent_factor_topologies': latent_factor_topologies,
    'rec_dist': 'bernoulli',    # Reconstruction distribution
    'log_kl_components': True,
#     'schedulers_kwargs':[
#     {
#         'name': 'linear',
#         'kwargs': {
#             'param_name': 'gamma',
#             'initial_value': 50,
#             'final_value': 200,
#             'total_steps': 100000
#         }
#     }
# ]
}

#### device parameters ####
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print(f"Using GPU")
else:
    print("Using CPU")

### Checkpoint parameters ###
return_chkpt = False
chkpt_every_n_steps = 2400

# chkpt_save_path = 'checkpoints/tests/test-epoch-1.pt'
chkpt_save_path = None

chkpt_save_dir = None
# chkpt_save_dir = 'checkpoints/tests_s_n_vae'

chkpt_save_master_dir = None
chkpt_viz = False

# Reproducibility

In [ ]:
if determinism_kwargs is not None:
    # MUST Be set before importing any other modules
    # to ensure reproducibility across all libraries
    from utils.reproducibility import set_deterministic_run, get_deterministic_dataloader
    set_deterministic_run(**determinism_kwargs)
    print(f"Set deterministic run with kwargs: {determinism_kwargs}")

# Imports

In [ ]:
import torch
import numpy as np
import torch.utils
import matplotlib.pyplot as plt
import torch.optim as optim

import utils.visualize
from trainers import UnsupervisedTrainer
import losses
import vae_models
from datasets import get_dataset
from utils.io import find_optimal_num_workers
from metrics.utils import MetricAggregator

## Setup Model, Loss, and Optimizer

In [ ]:
def setup_components(dataset, loss_kwargs, latent_factor_topologies):
    """Instantiates model, loss function, and optimizer based on config."""
    img_size = dataset[0][0].shape
    n_data = len(dataset)
    
    # Instantiate S-N-VAE Model
    model = vae_models.select(name=model_name, 
                              img_size=img_size, 
                              latent_factor_topologies=latent_factor_topologies,
                              decoder_output_dist=model_decoder_output_dist
                              ).to(device)

    # Instantiate S-N-Annealed-VAE Loss
    loss_fn = losses.select(loss_name, **loss_kwargs)

    # Instantiate Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"--- Setup for {dataset.__class__.__name__} --- ")
    print(f"Model: {model.model_name}")
    print(f"Latent factor topologies: {latent_factor_topologies}")
    print(f"Loss: {loss_fn.name} (rec_dist={loss_kwargs['rec_dist']}), kwargs={loss_kwargs}")
    print(f"Optimizer: {optimizer.__class__.__name__}")
    print(f"Learning rate: {learning_rate}")

    return model, loss_fn, optimizer

# dSprites

In [ ]:
# Load dSprites
Dsprites = get_dataset('dsprites')

# dsprites_dataset = Dsprites(selected_factors='all', not_selected_factors_index_value=None)
dsprites_dataset = Dsprites(selected_factors=['posX', 'posY', 'orientation'], not_selected_factors_index_value={'scale':5, 'shape':0, 'color':0})

# num_workers_dsprites = find_optimal_num_workers(dsprites_dataset, batch_size=batch_size, num_batches_to_test='all')
num_workers_dsprites = 7

if determinism_kwargs is not None:
    dsprites_dataloader = get_deterministic_dataloader(dataset=dsprites_dataset,
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   num_workers=num_workers_dsprites,
                                                   seed=determinism_kwargs['seed'],
                                                   pin_memory=True)
else:
    dsprites_dataloader = torch.utils.data.DataLoader(dsprites_dataset, 
                                                      batch_size=batch_size, 
                                                      num_workers=num_workers_dsprites, 
                                                      shuffle=True, 
                                                      pin_memory=True)

print(f"Loaded dSprites dataset with {len(dsprites_dataset)} samples.")

## Train

In [ ]:
print("\n===== Training S-N-Annealed-VAE on dSprites =====")
model_dsprites, loss_fn_dsprites, optimizer_dsprites = setup_components(dsprites_dataset, 
                                                                        loss_kwargs_dsprites,
                                                                        latent_factor_topologies
                                                                        )
trainer_dsprites = UnsupervisedTrainer(model=model_dsprites,
                                      loss=loss_fn_dsprites,
                                      optimizer=optimizer_dsprites,
                                      lr_scheduler=None,
                                      determinism_kwargs=determinism_kwargs,
                                      use_torch_compile=use_torch_compile,
                                      return_logs=True,
                                      return_chkpt=return_chkpt,
                                      chkpt_save_path=chkpt_save_path,
                                      chkpt_save_dir=chkpt_save_dir,
                                      chkpt_every_n_steps=chkpt_every_n_steps,
                                      chkpt_viz=chkpt_viz
                                      )

trainer_dsprites.train(max_steps=num_train_steps, step_unit=train_step_unit, dataloader=dsprites_dataloader)

In [ ]:
print("\n===== Visualizing dSprites Results =====")
visualizer_dsprites = utils.visualize.SNVAEVisualizer(vae_model=model_dsprites, dataset=dsprites_dataset)

## Visualization

### Reconstructions

In [ ]:
print("Plotting random reconstructions...")
visualizer_dsprites.plot_random_reconstructions(10, mode='mean')
plt.show()

print("Plotting reconstructions from specific indices...")
indices_dsprites = [0, 10, 20, 30, 40, 50]  # Example indices
visualizer_dsprites.plot_reconstructions_sub_dataset(indices_dsprites, mode='mean')
plt.show()

### Latent traversal

In [ ]:
print("Plotting latent traversals...")
visualizer_dsprites.plot_all_latent_traversals(num_samples=15,
                                               r1_max_traversal_type='probability',
                                               r1_max_traversal=0.95,
                                               s1_max_traversal_type='fraction',
                                               s1_max_traversal=1.0,
                                               use_ref_img=True
                                               )
plt.show()

In [ ]:
# Select a reference image index for dSprites
ref_img_idx_dsprites = 495  # Example index
ref_img_dsprites = dsprites_dataset[ref_img_idx_dsprites][0]

# Plot the reference image
plt.imshow(ref_img_dsprites.permute(1, 2, 0).cpu().numpy(), cmap='gray')
plt.title(f"dSprites Reference Image (Index: {ref_img_idx_dsprites})")
plt.show()

In [ ]:
# Single latent traversal based on the reference image
latent_factor_idx_dsprites = 0  # Index of the latent dimension to traverse
print(f"Plotting single latent traversal for dimension {latent_factor_idx_dsprites}...")
visualizer_dsprites.plot_single_latent_traversal(latent_factor_idx_dsprites, 
                                                 ref_img=ref_img_dsprites, 
                                                 num_samples=11,
                                                 max_traversal_type='fraction',
                                                 max_traversal=1)
plt.show()

In [ ]:
# All latent traversals based on the reference image
print("Plotting all latent traversals based on reference image...")
visualizer_dsprites.plot_all_latent_traversals(ref_img=ref_img_dsprites, 
                                               num_samples=15,
                                               r1_max_traversal_type='probability',
                                               r1_max_traversal=0.95,
                                               s1_max_traversal_type='fraction',
                                               s1_max_traversal=1
                                               )

# G-Commutative VAE Extension

In this section, we extend the S-N-Annealed-VAE model with Group Theory losses, specifically implementing g-commutative constraints. The g-commutative loss enforces commutativity properties in the latent space transformations, encouraging the model to learn more disentangled and structured representations.

## Design Philosophy: Controlled Comparison
**All basic training settings are kept identical** to the standard S-N-Annealed-VAE to ensure fair comparison. The **only difference** is the addition of group theory constraints on top of the base loss.

## What Remains Identical:
- ✅ **Model Architecture**: Same S-N-VAE with Locatello encoder/decoder
- ✅ **Reconstruction Loss**: Same Bernoulli distribution and annealed capacity weighting  
- ✅ **Training Parameters**: Same learning rate, batch size, epochs, optimizer
- ✅ **Latent Topologies**: Same ['S1', 'R1', 'R1'] factor configuration
- ✅ **Base Loss Function**: Identical Annealed S-N-VAE loss as foundation

## What Is Added (Group Theory Extensions):
- 🔄 **Commutative Constraints**: Enforces g∘g' = g'∘g in latent space
- 📈 **Progressive Scheduling**: Gradually increases constraint strength during training  
- ⏰ **Warm-up Period**: First 5000 steps use only base loss for stability
- 🎯 **Group Actions**: R¹ (translation) and S¹ (rotation) transformations

This design ensures that any performance differences can be directly attributed to the group theory constraints rather than changes in basic training setup.

# G-Commutative Hyperparameters

In [ ]:
#### G-Commutative Loss Configuration ####
# DESIGN PHILOSOPHY: Keep ALL basic training settings identical to standard S-N-Annealed-VAE
# The ONLY difference is the addition of group theory constraints on top of the base loss

# FIXED: Use correct loss name for S-N-VAE group theory
g_commutative_loss_name = 'group_theory_snvae'  # S-N-VAE compatible group theory loss

# Base loss configuration - use EXACT same settings as standard S-N-Annealed-VAE
base_loss_g_commutative = {
    'name': 'anneal_s_n_vae',
    'kwargs': loss_kwargs_dsprites  # Identical to standard S-N-Annealed-VAE settings
}

# Group Theory Loss Parameters for G-Commutative
loss_kwargs_g_commutative = {
    'base_loss_name': base_loss_g_commutative['name'],
    'base_loss_kwargs': base_loss_g_commutative['kwargs'],
    'latent_factor_topologies': latent_factor_topologies,
    'device': device,  # FIXED: Required parameter
    
    # KEEP IDENTICAL: Same reconstruction distribution as standard S-N-Annealed-VAE
    'rec_dist': 'bernoulli',  # Same as standard S-N-Annealed-VAE for fair comparison
    
    ### ADDED GROUP THEORY CONSTRAINTS (only difference) ###
    'commutative_weight': 1.0,                    # Enable commutative loss
    'commutative_component_order': 2,             # Use pairs for commutative operations
    'commutative_comparison_dist': 'gaussian',    # Comparison metric for commutative constraints
    
    ### Meaningful settings (disabled for pure g-commutative) ###
    'meaningful_weight': 0.0,                     # Disable meaningful loss for pure commutative
    'meaningful_component_order': 1,
    'meaningful_transformation_order': 1,
    'meaningful_critic_gradient_penalty_weight': 10.0,  # FIXED: Required parameter (even if not used)
    'meaningful_critic_lr': 1e-4,                 # FIXED: Required parameter (even if not used)
    'meaningful_n_critic': 1,
    
    ### Group theory general settings ###
    'deterministic_rep': True,                    # Use deterministic representations  
    'g_action_r1_range': 2.0,                    # Range for R1 (translation) actions
    'g_action_s1_range': 2 * torch.pi,           # Full rotation range for S1 actions
    'g_action_r1_dist': 'uniform',               # Distribution for R1 action sampling
    'g_action_s1_dist': 'uniform',               # Distribution for S1 action sampling
    'comp_latent_select_threshold': 0.1,         # Threshold for selecting latent components
    'warm_up_steps': 5000,                       # Warm-up steps before applying group losses
    
    ### Learning rate scheduling ###
    'schedulers_kwargs': [
        {
            'name': 'linear',
            'kwargs': {
                'param_name': 'commutative_weight',
                'initial_value': 0.05,           # Start with low commutative weight
                'final_value': 10.0,             # Gradually increase to strong constraint
                'before_start_value': 0,
                'start_step': 5000,              # Start scheduling after warm-up
                'total_steps': int(num_train_steps * 0.8 if train_step_unit == 'epoch' 
                                 else num_train_steps * 0.8)  # Schedule over 80% of training
            }
        }
    ]
}

print("=== TRAINING SETTINGS COMPARISON ===")
print("Standard S-N-Annealed-VAE vs G-Commutative S-N-Annealed-VAE:")
print(f"✓ Model architecture: IDENTICAL ({model_name})")
print(f"✓ Decoder output dist: IDENTICAL ({model_decoder_output_dist})")
print(f"✓ Latent topologies: IDENTICAL ({latent_factor_topologies})")
print(f"✓ Learning rate: IDENTICAL ({learning_rate})")
print(f"✓ Batch size: IDENTICAL ({batch_size})")
print(f"✓ Training steps: IDENTICAL ({num_train_steps} {train_step_unit}s)")
print(f"✓ Base loss: IDENTICAL ({base_loss_g_commutative['name']} with bernoulli)")
print(f"✓ Base loss params: IDENTICAL (gamma={loss_kwargs_dsprites['gamma']}, C_fin={loss_kwargs_dsprites['C_fin']})")
print(f"+ ADDITIONAL: Group theory commutative constraints (weight={loss_kwargs_g_commutative['commutative_weight']})")
print(f"+ ADDITIONAL: Warm-up period ({loss_kwargs_g_commutative['warm_up_steps']} steps)")
print(f"🔧 FIXED: Using correct loss name '{g_commutative_loss_name}' for S-N-VAE")
print("\n→ This ensures fair comparison: same base training + group constraints")

## Setup Model, Loss, and Optimizer for G-Commutative

In [ ]:
def setup_g_commutative_components(dataset, loss_kwargs, latent_factor_topologies):
    """Instantiates model, group theory loss function, and optimizer for g-commutative training."""
    img_size = dataset[0][0].shape
    n_data = len(dataset)
    
    # Instantiate S-N-VAE Model (same as before - the model architecture doesn't change)
    model = vae_models.select(name=model_name, 
                              img_size=img_size, 
                              latent_factor_topologies=latent_factor_topologies,
                              decoder_output_dist=model_decoder_output_dist
                              ).to(device)

    # Instantiate Group Theory Loss with G-Commutative constraints
    loss_fn = losses.select(g_commutative_loss_name, **loss_kwargs)

    # Instantiate Optimizer (same as before)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"--- G-Commutative Setup for {dataset.__class__.__name__} --- ")
    print(f"Model: {model.model_name}")
    print(f"Latent factor topologies: {latent_factor_topologies}")
    print(f"Loss: {loss_fn.name}")
    print(f"  ↳ Base loss: {loss_kwargs['base_loss_name']}")
    print(f"  ↳ Commutative weight: {loss_kwargs['commutative_weight']}")
    print(f"  ↳ Meaningful weight: {loss_kwargs['meaningful_weight']}")
    print(f"  ↳ Reconstruction dist: {loss_kwargs['rec_dist']}")
    print(f"Optimizer: {optimizer.__class__.__name__}")
    print(f"Learning rate: {learning_rate}")

    return model, loss_fn, optimizer

## Train G-Commutative Model

In [ ]:
print("\n===== Training G-Commutative S-N-Annealed-VAE on dSprites =====")
model_g_comm_dsprites, loss_fn_g_comm_dsprites, optimizer_g_comm_dsprites = setup_g_commutative_components(
    dsprites_dataset, 
    loss_kwargs_g_commutative,
    latent_factor_topologies
)

# Setup trainer for G-Commutative model
trainer_g_comm_dsprites = UnsupervisedTrainer(model=model_g_comm_dsprites,
                                             loss=loss_fn_g_comm_dsprites,
                                             optimizer=optimizer_g_comm_dsprites,
                                             lr_scheduler=None,
                                             determinism_kwargs=determinism_kwargs,
                                             use_torch_compile=use_torch_compile,
                                             return_logs=True,
                                             return_chkpt=return_chkpt,
                                             chkpt_save_path=chkpt_save_path,
                                             chkpt_save_dir=chkpt_save_dir,
                                             chkpt_every_n_steps=chkpt_every_n_steps,
                                             chkpt_viz=chkpt_viz
                                             )

# Train the G-Commutative model
print(f"\nStarting G-Commutative training for {num_train_steps} {train_step_unit}s...")
print(f"Note: Group losses will be applied after {loss_kwargs_g_commutative['warm_up_steps']} warm-up steps")
trainer_g_comm_dsprites.train(max_steps=num_train_steps, step_unit=train_step_unit, dataloader=dsprites_dataloader)

In [ ]:
print("\n===== Visualizing G-Commutative dSprites Results =====")
visualizer_g_comm_dsprites = utils.visualize.SNVAEVisualizer(vae_model=model_g_comm_dsprites, dataset=dsprites_dataset)

## G-Commutative Visualization

### G-Commutative Reconstructions

In [ ]:
print("Plotting G-Commutative random reconstructions...")
visualizer_g_comm_dsprites.plot_random_reconstructions(10, mode='mean')
plt.show()

print("Plotting G-Commutative reconstructions from specific indices...")
indices_g_comm_dsprites = [0, 10, 20, 30, 40, 50]  # Same indices for comparison
visualizer_g_comm_dsprites.plot_reconstructions_sub_dataset(indices_g_comm_dsprites, mode='mean')
plt.show()

### G-Commutative Latent Traversal

In [ ]:
print("Plotting G-Commutative latent traversals...")
visualizer_g_comm_dsprites.plot_all_latent_traversals(num_samples=15,
                                                      r1_max_traversal_type='probability',
                                                      r1_max_traversal=0.95,
                                                      s1_max_traversal_type='fraction',
                                                      s1_max_traversal=1.0,
                                                      use_ref_img=True
                                                      )
plt.show()

In [ ]:
# Use the same reference image for direct comparison with standard S-N-Annealed-VAE results
print(f"Using the same reference image (Index: {ref_img_idx_dsprites}) for G-Commutative traversals...")

# Plot the reference image again for context
plt.figure(figsize=(4, 4))
plt.imshow(ref_img_dsprites.permute(1, 2, 0).cpu().numpy(), cmap='gray')
plt.title(f"G-Commutative Reference Image (Index: {ref_img_idx_dsprites})")
plt.axis('off')
plt.show()

# Single latent traversal with G-Commutative model
latent_factor_idx_g_comm = 0  # Same dimension as before
print(f"Plotting G-Commutative single latent traversal for dimension {latent_factor_idx_g_comm}...")
visualizer_g_comm_dsprites.plot_single_latent_traversal(latent_factor_idx_g_comm, 
                                                        ref_img=ref_img_dsprites, 
                                                        num_samples=11,
                                                        max_traversal_type='fraction',
                                                        max_traversal=1)
plt.show()

In [ ]:
# All latent traversals with G-Commutative model using the reference image
print("Plotting all G-Commutative latent traversals based on reference image...")
visualizer_g_comm_dsprites.plot_all_latent_traversals(ref_img=ref_img_dsprites, 
                                                      num_samples=15,
                                                      r1_max_traversal_type='probability',
                                                      r1_max_traversal=0.95,
                                                      s1_max_traversal_type='fraction',
                                                      s1_max_traversal=1
                                                      )
plt.show()